# Functions

In [1]:
import pandas as pd
import requests
from typing import Dict, List
import re
import json

def get_github_folder_files(username: str, repo_name: str, folder_path: str,
                           file_extension: str = ".csv") -> List[str]:
    """
    GitHub API를 사용해서 특정 폴더의 파일 목록을 가져오는 함수

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")
    file_extension : str
        필터링할 파일 확장자 (기본값: ".csv")

    Returns:
    --------
    List[str]
        해당 폴더의 CSV 파일명 목록
    """

    # GitHub API URL
    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{folder_path}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()

        files_data = response.json()

        # CSV 파일만 필터링
        csv_files = []
        for file_info in files_data:
            if file_info['type'] == 'file' and file_info['name'].endswith(file_extension):
                csv_files.append(file_info['name'])

        return sorted(csv_files)

    except requests.exceptions.RequestException as e:
        print(f"❌ GitHub API 요청 실패: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: {e}")
        return []


def create_variable_name_from_filename(filename: str) -> str:
    """
    파일명에서 변수명을 생성하는 함수
    다양한 temp 패턴을 지원: temp2_1, temp2_2, temp1 등

    Parameters:
    -----------
    filename : str
        파일명 (예: "temp2_1_governor_busan_3.csv", "temp2_2_governor_seoul_4.csv")

    Returns:
    --------
    str
        변수명 (예: "g3_busan", "g4_seoul")
    """

    # temp2_X_governor_지역명_숫자.csv → g숫자_지역명 (X는 1, 2, 3... 등 어떤 숫자든)
    match = re.search(r'temp2_\d+_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # temp1_governor_지역명_숫자.csv → g숫자_지역명 (기존 패턴 지원)
    match = re.search(r'temp1_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # 일반적인 tempX_Y_governor 패턴 (더 확장 가능)
    match = re.search(r'temp\d+_\d+_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # 매칭되지 않으면 기본적으로 파일명 정리해서 반환
    base_name = filename.replace('.csv', '')
    base_name = re.sub(r'temp\d+_\d*_?', '', base_name)  # tempX_Y_ 또는 tempX_ 제거
    return base_name


def load_all_csvs_from_github_folder(username: str, repo_name: str, folder_path: str) -> Dict[str, pd.DataFrame]:
    """
    GitHub 폴더에서 모든 CSV 파일을 자동으로 읽어와서 DataFrame 딕셔너리로 반환

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름 (예: "korean-elections")
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")

    Returns:
    --------
    Dict[str, pd.DataFrame]
        변수명을 키로 하고 DataFrame을 값으로 하는 딕셔너리
    """

    # 1. GitHub API로 파일 목록 가져오기
    csv_files = get_github_folder_files(username, repo_name, folder_path)

    if not csv_files:
        print("❌ CSV 파일을 찾을 수 없습니다.")
        return {}

    # 2. 각 CSV 파일 읽어오기
    base_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/refs/heads/main/{folder_path}/"
    dataframes = {}

    for filename in csv_files:
        try:
            # URL 생성
            url = base_url + filename

            # 변수명 생성
            var_name = create_variable_name_from_filename(filename)

            # CSV 읽기
            df = pd.read_csv(url)

            dataframes[var_name] = df
            print(f"✅ {filename} → {var_name}")

        except Exception as e:
            print(f"❌ {filename} 읽기 실패: {e}")

    return dataframes


def create_individual_variables(dataframes: Dict[str, pd.DataFrame], globals_dict):
    """
    딕셔너리로 저장된 DataFrame들을 개별 변수로 생성

    Parameters:
    -----------
    dataframes : Dict[str, pd.DataFrame]
        load_all_csvs_from_github_folder()의 반환값
    globals_dict : dict
        변수를 저장할 전역 네임스페이스 (globals()를 전달)
    """

    for var_name, df in dataframes.items():
        globals_dict[var_name] = df
        print(f"변수 생성: {var_name}")


def extract_and_combine_csvs(username: str, repo_name: str, folder_path: str,
                           filter_column: str, filter_value: str,
                           new_column, new_column_value) -> pd.DataFrame:
    """
    GitHub 폴더의 모든 CSV 파일에서 특정 조건의 행들을 추출하여 하나로 합치는 함수

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름
    folder_path : str
        폴더 경로 (예: "temp/v2_1_g/3rd_2002")
    filter_column : str
        필터링할 열 이름
    filter_value : str
        추출할 행의 해당 열 값
    new_column : str or List[str]
        새로 생성할 열 이름(들)
    new_column_value : str or List[str]
        새로 생성할 열의 값(들) - new_column과 같은 길이여야 함

    Returns:
    --------
    pd.DataFrame
        조건에 맞는 모든 행들이 합쳐진 DataFrame
    """

    # 1. GitHub API로 파일 목록 가져오기
    csv_files = get_github_folder_files(username, repo_name, folder_path)

    if not csv_files:
        print("❌ CSV 파일을 찾을 수 없습니다.")
        return pd.DataFrame()

    print(f"📋 발견된 파일들: {len(csv_files)}개")

    # 2. 각 CSV 파일에서 조건에 맞는 행들 추출하여 리스트에 저장
    base_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/refs/heads/main/{folder_path}/"
    extracted_dataframes = []

    for filename in csv_files:
        try:
            # URL 생성 및 CSV 읽기
            url = base_url + filename
            df = pd.read_csv(url)

            # 필터 조건에 맞는 행들 추출
            if filter_column in df.columns:
                filtered_df = df[df[filter_column] == filter_value].copy()

                if not filtered_df.empty:
                    extracted_dataframes.append(filtered_df)
                    print(f"✅ {filename}: {len(filtered_df)}행 추출")
                else:
                    print(f"⚠️ {filename}: 조건에 맞는 행 없음")
            else:
                print(f"❌ {filename}: '{filter_column}' 열이 존재하지 않음")

        except Exception as e:
            print(f"❌ {filename} 처리 실패: {e}")

    # 3. 모든 추출된 DataFrame들을 아래로 차례차례 붙이기
    if not extracted_dataframes:
        print("❌ 추출된 데이터가 없습니다.")
        return pd.DataFrame()

    combined_df = pd.concat(extracted_dataframes, ignore_index=True)

    # 4. 새로운 열(들) 추가
    # 단일 값인 경우와 리스트인 경우 모두 처리
    if isinstance(new_column, list) and isinstance(new_column_value, list):
        # 리스트로 입력된 경우
        if len(new_column) != len(new_column_value):
            print("❌ new_column과 new_column_value의 길이가 다릅니다.")
            return combined_df

        for col_name, col_value in zip(new_column, new_column_value):
            combined_df[col_name] = col_value
            print(f"📊 새로운 열 '{col_name}' 추가 완료 (값: '{col_value}')")

    else:
        # 단일 값으로 입력된 경우 (기존 방식)
        combined_df[new_column] = new_column_value
        print(f"📊 새로운 열 '{new_column}' 추가 완료 (값: '{new_column_value}')")

    print(f"\n🎉 총 {len(combined_df)}행이 결합되었습니다.")

    return combined_df


# # 사용 예시
# def example_usage():
#     """사용 예시"""

#     print("=== GitHub CSV 추출 및 결합 함수 ===\n")

#     print("1️⃣ 기본 CSV 로더:")
#     print("""
#     username = "your-username"
#     repo_name = "korean-elections"
#     folder_path = "temp/v1_g/3rd_2002"

#     # 모든 CSV 파일을 자동으로 읽어와서 딕셔너리로 저장
#     election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

#     # 방법 1: 딕셔너리로 접근
#     busan_df = election_data['g3_busan']
#     seoul_df = election_data['g3_seoul']

#     # 방법 2: 개별 변수로 생성
#     create_individual_variables(election_data, globals())
#     """)

#     print("\n2️⃣ 새로운 추출 및 결합 함수:")
#     print("""
#     # 단일 열 추가 (기존 방식)
#     result_df = extract_and_combine_csvs(
#         username="your-username",
#         repo_name="korean-elections",
#         folder_path="temp/v2_1_g/3rd_2002",
#         filter_column="당선여부",        # 추출할 조건의 열 이름
#         filter_value="당선",            # 그 열에서 찾을 값
#         new_column="선거회차",          # 새로 만들 열 이름
#         new_column_value="3회"         # 새 열의 값
#     )

#     # 🆕 여러 열 동시 추가 (리스트 방식)
#     result_df = extract_and_combine_csvs(
#         username="your-username",
#         repo_name="korean-elections",
#         folder_path="temp/v2_1_g/3rd_2002",
#         filter_column="당선여부",
#         filter_value="당선",
#         new_column=["선거회차", "데이터출처", "분석날짜"],        # 리스트로 여러 열 이름
#         new_column_value=["3회", "선관위", "2025-07-24"]       # 각 열에 대응되는 값들
#     )
#     """)

#     print("\n3️⃣ 여러 회차 데이터 결합:")
#     print("""
#     # 여러 회차의 당선자 데이터를 하나로 합치기 (각각 다른 정보 추가)
#     winners_3rd = extract_and_combine_csvs(
#         username, repo_name, "temp/v2_1_g/3rd_2002",
#         "당선여부", "당선",
#         ["회차", "년도", "구분"],
#         ["3회", "2002", "지방선거"]
#     )

#     winners_4th = extract_and_combine_csvs(
#         username, repo_name, "temp/v2_1_g/4th_2006",
#         "당선여부", "당선",
#         ["회차", "년도", "구분"],
#         ["4회", "2006", "지방선거"]
#     )

#     # 두 회차 데이터를 최종 결합
#     all_winners = pd.concat([winners_3rd, winners_4th], ignore_index=True)
#     """)

#     print("\n4️⃣ 실용적인 활용 예시:")
#     print("""
#     # 특정 정당의 후보들만 추출하면서 추가 정보 기록
#     democratic_candidates = extract_and_combine_csvs(
#         username, repo_name, "temp/v2_1_g/3rd_2002",
#         filter_column="정당",
#         filter_value="더불어민주당",
#         new_column=["추출조건", "분석자", "분석일시"],
#         new_column_value=["더불어민주당_후보", "연구팀", "2025-07-24"]
#     )
#     """)


# if __name__ == "__main__":
#     example_usage()

# Preprocessing

## 3rd_2002

In [2]:
# 예시: 모든 CSV에서 '구시군' 열의 값이 '합계'인 행들만 추출하여 결계
df_3 = extract_and_combine_csvs(
    username="sw1kwon",
    repo_name="korean-elections",
    folder_path="temp/v2_2_g/3rd_2002",
    filter_column="구시군",        # 추출할 조건의 열 이름
    filter_value="합계",            # 그 열에서 찾을 값
    new_column=["선거종류", "선거년도"],        # 리스트
    new_column_value=["광역단체장", "2002"]       # 리스트 (같은 인덱스 대응)
)

📋 발견된 파일들: 16개
✅ temp2_2_governor_busan_3.csv: 1행 추출
✅ temp2_2_governor_chungbuk_3.csv: 1행 추출
✅ temp2_2_governor_chungnam_3.csv: 1행 추출
✅ temp2_2_governor_daegu_3.csv: 1행 추출
✅ temp2_2_governor_daejeon_3.csv: 1행 추출
✅ temp2_2_governor_gangwon_3.csv: 1행 추출
✅ temp2_2_governor_gwangju_3.csv: 1행 추출
✅ temp2_2_governor_gyeongbuk_3.csv: 1행 추출
✅ temp2_2_governor_gyeonggi_3.csv: 1행 추출
✅ temp2_2_governor_gyeongnam_3.csv: 1행 추출
✅ temp2_2_governor_incheon_3.csv: 1행 추출
✅ temp2_2_governor_jeju_3.csv: 1행 추출
✅ temp2_2_governor_jeonbuk_3.csv: 1행 추출
✅ temp2_2_governor_jeonnam_3.csv: 1행 추출
✅ temp2_2_governor_seoul_3.csv: 1행 추출
✅ temp2_2_governor_ulsan_3.csv: 1행 추출
📊 새로운 열 '선거종류' 추가 완료 (값: '광역단체장')
📊 새로운 열 '선거년도' 추가 완료 (값: '2002')

🎉 총 16행이 결합되었습니다.


In [3]:
df_3

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,부산광역시,합계,2784721,1163040,729589,221938,192594,0,1144121,18919,1621681,광역단체장,2002
1,충청북도,합계,1076451,600550,343546,0,196253,46459,586258,14292,475901,광역단체장,2002
2,충청남도,합계,1397105,785742,251005,0,508796,0,759801,25941,611363,광역단체장,2002
3,대구광역시,합계,1814278,751994,452943,0,0,287396,740339,11655,1062284,광역단체장,2002
4,대전광역시,합계,987180,417809,191832,0,165426,54225,411483,6326,569371,광역단체장,2002
5,강원도,합계,1129859,671242,468987,190451,0,0,659438,11804,458617,광역단체장,2002
6,광주광역시,합계,954481,403821,43695,185938,29427,138088,397148,6673,550660,광역단체장,2002
7,경상북도,합계,2044215,1234095,1028080,0,0,174472,1202552,31543,810120,광역단체장,2002
8,경기도,합계,6777575,3024844,1744291,1075243,168357,0,2987891,36953,3752731,광역단체장,2002
9,경상남도,합계,2227548,1259142,920706,208641,106367,0,1235714,23428,968406,광역단체장,2002


In [4]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      16 non-null     object
 1   구시군     16 non-null     object
 2   선거인수    16 non-null     int64 
 3   투표수     16 non-null     int64 
 4   보수정당    16 non-null     int64 
 5   진보정당    16 non-null     int64 
 6   그외정당    16 non-null     int64 
 7   무소속     16 non-null     int64 
 8   득표수_계   16 non-null     int64 
 9   무효투표수   16 non-null     int64 
 10  기권수     16 non-null     int64 
 11  선거종류    16 non-null     object
 12  선거년도    16 non-null     object
dtypes: int64(9), object(4)
memory usage: 1.8+ KB


In [5]:
# 수치형 열만 합계 구하기
summary_row = df_3.select_dtypes(include='number').sum().to_frame().T

# object 열 값 추가
summary_row.insert(0, '시도', '전국')
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '선거종류', '광역단체장')
summary_row.insert(0, '선거년도', '2002')


# summary_row를 맨 아래에 붙이기
df_3 = pd.concat([df_3, summary_row], ignore_index=True)

In [6]:
df_3['시도'].unique()

array(['부산광역시', '충청북도', '충청남도', '대구광역시', '대전광역시', '강원도', '광주광역시', '경상북도',
       '경기도', '경상남도', '인천광역시', '제주도', '전라북도', '전라남도', '서울특별시', '울산광역시',
       '전국'], dtype=object)

In [7]:
import pandas as pd

# 1. 지역명 매핑 딕셔너리 정의
region_mapping = {
    '부산광역시': '부산',
    '충청북도': '충북',
    '충청남도': '충남',
    '대구광역시': '대구',
    '대전광역시': '대전',
    '강원도': '강원',
    '광주광역시': '광주',
    '경상북도': '경북',
    '경기도': '경기',
    '경상남도': '경남',
    '인천광역시': '인천',
    '제주도': '제주',
    '전라북도': '전북',
    '전라남도': '전남',
    '서울특별시': '서울',
    '울산광역시': '울산',
    '전국': '전국'
}

# 2. 지역명 변경
df_3 = df_3.replace({'시도': region_mapping})

# 3. 원하는 행 순서 정의
desired_order = [
    '전국', '서울', '부산', '대구', '인천', '광주', '대전', '울산',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

# 4. 행 순서 변경
df_3 = df_3.set_index('시도').reindex(desired_order).reset_index()

In [8]:
df_3 = df_3.rename(columns={'시도': '지역'}).drop(columns=['구시군']).reset_index(drop=True)

In [9]:
df_3

,지역,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,전국,34744232,16973662,8820102,4874653,1761560,1226757,16683072,290590,17770570,광역단체장,2002
1,서울,7665343,3510898,1819057,1496754,128981,34313,3479105,31793,4154445,광역단체장,2002
2,부산,2784721,1163040,729589,221938,192594,0,1144121,18919,1621681,광역단체장,2002
3,대구,1814278,751994,452943,0,0,287396,740339,11655,1062284,광역단체장,2002
4,인천,1809907,711602,393932,225210,82111,0,701253,10349,1098305,광역단체장,2002
5,광주,954481,403821,43695,185938,29427,138088,397148,6673,550660,광역단체장,2002
6,대전,987180,417809,191832,0,165426,54225,411483,6326,569371,광역단체장,2002
7,울산,722806,378121,197772,0,174875,0,372647,5474,344685,광역단체장,2002
8,경기,6777575,3024844,1744291,1075243,168357,0,2987891,36953,3752731,광역단체장,2002
9,강원,1129859,671242,468987,190451,0,0,659438,11804,458617,광역단체장,2002


## 4th_2006

In [10]:
# 예시: 모든 CSV에서 '구시군' 열의 값이 '합계'인 행들만 추출하여 결계
df_4 = extract_and_combine_csvs(
    username="sw1kwon",
    repo_name="korean-elections",
    folder_path="temp/v2_2_g/4th_2006",
    filter_column="구시군",        # 추출할 조건의 열 이름
    filter_value="합계",            # 그 열에서 찾을 값
    new_column=["선거종류", "선거년도"],        # 리스트
    new_column_value=["광역단체장", "2006"]       # 리스트 (같은 인덱스 대응)
)

📋 발견된 파일들: 16개
✅ temp2_2_governor_busan_4.csv: 1행 추출
✅ temp2_2_governor_chungbuk_4.csv: 1행 추출
✅ temp2_2_governor_chungnam_4.csv: 1행 추출
✅ temp2_2_governor_daegu_4.csv: 1행 추출
✅ temp2_2_governor_daejeon_4.csv: 1행 추출
✅ temp2_2_governor_gangwon_4.csv: 1행 추출
✅ temp2_2_governor_gwangju_4.csv: 1행 추출
✅ temp2_2_governor_gyeongbuk_4.csv: 1행 추출
✅ temp2_2_governor_gyeonggi_4.csv: 1행 추출
✅ temp2_2_governor_gyeongnam_4.csv: 1행 추출
✅ temp2_2_governor_incheon_4.csv: 1행 추출
✅ temp2_2_governor_jeju_4.csv: 1행 추출
✅ temp2_2_governor_jeonbuk_4.csv: 1행 추출
✅ temp2_2_governor_jeonnam_4.csv: 1행 추출
✅ temp2_2_governor_seoul_4.csv: 1행 추출
✅ temp2_2_governor_ulsan_4.csv: 1행 추출
📊 새로운 열 '선거종류' 추가 완료 (값: '광역단체장')
📊 새로운 열 '선거년도' 추가 완료 (값: '2006')

🎉 총 16행이 결합되었습니다.


In [11]:
df_4

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,부산광역시,합계,2845104,1378618,895214,329470,141061,0,1365745,12873,1466486,광역단체장,2006
1,충청북도,합계,1126282,616053,361157,185426,58741,0,605324,10729,510229,광역단체장,2006
2,충청남도,합계,1503240,838462,379420,178169,261671,0,819260,19202,664778,광역단체장,2006
3,대구광역시,합계,1885043,915060,636057,191131,44261,35232,906681,8379,969983,광역단체장,2006
4,대전광역시,합계,1077468,532568,231489,217273,79279,0,528041,4527,544900,광역단체장,2006
5,강원도,합계,1160977,681633,471613,148302,48411,0,668326,13307,479344,광역단체장,2006
6,광주광역시,합계,1014620,469316,18461,157756,288501,0,464718,4598,545304,광역단체장,2006
7,경상북도,합계,2087709,1284342,961363,290358,0,0,1251721,32621,803367,광역단체장,2006
8,경기도,합계,7918828,3695552,2181677,1124317,349515,0,3655509,40043,4223276,광역단체장,2006
9,경상남도,합계,2375265,1373815,852377,343137,154729,0,1350243,23572,1001450,광역단체장,2006


In [12]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      16 non-null     object
 1   구시군     16 non-null     object
 2   선거인수    16 non-null     int64 
 3   투표수     16 non-null     int64 
 4   보수정당    16 non-null     int64 
 5   진보정당    16 non-null     int64 
 6   그외정당    16 non-null     int64 
 7   무소속     16 non-null     int64 
 8   득표수_계   16 non-null     int64 
 9   무효투표수   16 non-null     int64 
 10  기권수     16 non-null     int64 
 11  선거종류    16 non-null     object
 12  선거년도    16 non-null     object
dtypes: int64(9), object(4)
memory usage: 1.8+ KB


In [13]:
# 수치형 열만 합계 구하기
summary_row = df_4.select_dtypes(include='number').sum().to_frame().T

# object 열 값 추가
summary_row.insert(0, '시도', '전국')
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '선거종류', '광역단체장')
summary_row.insert(0, '선거년도', '2006')


# summary_row를 맨 아래에 붙이기
df_4 = pd.concat([df_4, summary_row], ignore_index=True)

In [14]:
df_4['시도'].unique()

array(['부산광역시', '충청북도', '충청남도', '대구광역시', '대전광역시', '강원도', '광주광역시', '경상북도',
       '경기도', '경상남도', '인천광역시', '제주특별자치도', '전라북도', '전라남도', '서울특별시',
       '울산광역시', '전국'], dtype=object)

In [15]:
import pandas as pd

# 1. 지역명 매핑 딕셔너리 정의
region_mapping = {
    '부산광역시': '부산',
    '충청북도': '충북',
    '충청남도': '충남',
    '대구광역시': '대구',
    '대전광역시': '대전',
    '강원도': '강원',
    '광주광역시': '광주',
    '경상북도': '경북',
    '경기도': '경기',
    '경상남도': '경남',
    '인천광역시': '인천',
    '제주특별자치도': '제주',
    '전라북도': '전북',
    '전라남도': '전남',
    '서울특별시': '서울',
    '울산광역시': '울산',
    '전국': '전국'
}

# 2. 지역명 변경
df_4 = df_4.replace({'시도': region_mapping})

# 3. 원하는 행 순서 정의
desired_order = [
    '전국', '서울', '부산', '대구', '인천', '광주', '대전', '울산',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

# 4. 행 순서 변경
df_4 = df_4.set_index('시도').reindex(desired_order).reset_index()

In [16]:
df_4 = df_4.rename(columns={'시도': '지역'}).drop(columns=['구시군']).reset_index(drop=True)

In [17]:
df_4

,지역,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,전국,37064282,19118177,10418021,5106984,3166317,166284,18857606,260571,17946105,광역단체장,2006
1,서울,7983648,3977842,2409760,1077890,445368,13808,3946826,31016,4005806,광역단체장,2006
2,부산,2845104,1378618,895214,329470,141061,0,1365745,12873,1466486,광역단체장,2006
3,대구,1885043,915060,636057,191131,44261,35232,906681,8379,969983,광역단체장,2006
4,인천,1940403,859506,526932,200650,123237,0,850819,8687,1080897,광역단체장,2006
5,광주,1014620,469316,18461,157756,288501,0,464718,4598,545304,광역단체장,2006
6,대전,1077468,532568,231489,217273,79279,0,528041,4527,544900,광역단체장,2006
7,울산,790289,417200,261361,47579,104384,0,413324,3876,373089,광역단체장,2006
8,경기,7918828,3695552,2181677,1124317,349515,0,3655509,40043,4223276,광역단체장,2006
9,강원,1160977,681633,471613,148302,48411,0,668326,13307,479344,광역단체장,2006


## 5th_2010

In [18]:
# 예시: 모든 CSV에서 '구시군' 열의 값이 '합계'인 행들만 추출하여 결계
df_5 = extract_and_combine_csvs(
    username="sw1kwon",
    repo_name="korean-elections",
    folder_path="temp/v2_2_g/5th_2010",
    filter_column="구시군",        # 추출할 조건의 열 이름
    filter_value="합계",            # 그 열에서 찾을 값
    new_column=["선거종류", "선거년도"],        # 리스트
    new_column_value=["광역단체장", "2010"]       # 리스트 (같은 인덱스 대응)
)

📋 발견된 파일들: 16개
✅ temp2_2_governor_busan_5.csv: 1행 추출
✅ temp2_2_governor_chungbuk_5.csv: 1행 추출
✅ temp2_2_governor_chungnam_5.csv: 1행 추출
✅ temp2_2_governor_daegu_5.csv: 1행 추출
✅ temp2_2_governor_daejeon_5.csv: 1행 추출
✅ temp2_2_governor_gangwon_5.csv: 1행 추출
✅ temp2_2_governor_gwangju_5.csv: 1행 추출
✅ temp2_2_governor_gyeongbuk_5.csv: 1행 추출
✅ temp2_2_governor_gyeonggi_5.csv: 1행 추출
✅ temp2_2_governor_gyeongnam_5.csv: 1행 추출
✅ temp2_2_governor_incheon_5.csv: 1행 추출
✅ temp2_2_governor_jeju_5.csv: 1행 추출
✅ temp2_2_governor_jeonbuk_5.csv: 1행 추출
✅ temp2_2_governor_jeonnam_5.csv: 1행 추출
✅ temp2_2_governor_seoul_5.csv: 1행 추출
✅ temp2_2_governor_ulsan_5.csv: 1행 추출
📊 새로운 열 '선거종류' 추가 완료 (값: '광역단체장')
📊 새로운 열 '선거년도' 추가 완료 (값: '2010')

🎉 총 16행이 결합되었습니다.


In [19]:
df_5

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,부산광역시,합계,2849895,1410126,770507,619565,0,0,1390072,20054,1439769,광역단체장,2010
1,충청북도,합계,1183811,696393,313646,349913,19551,0,683110,13283,487418,광역단체장,2010
2,충청남도,합계,1595587,901863,154723,367288,347265,0,869271,32587,693724,광역단체장,2010
3,대구광역시,합계,1928835,886035,633118,146458,88599,0,868175,17860,1042800,광역단체장,2010
4,대전광역시,합계,1127547,596683,168616,137751,285196,0,591563,5120,530864,광역단체장,2010
5,강원도,합계,1190509,741724,326111,388443,0,0,714554,27170,448785,광역단체장,2010
6,광주광역시,합계,1064913,529901,74490,297003,151990,0,523483,6418,535012,광역단체장,2010
7,경상북도,합계,2122905,1260591,913812,143347,155361,0,1212520,48071,862314,광역단체장,2010
8,경기도,합계,8761840,4534771,2271492,0,2079892,0,4351384,183387,4227069,광역단체장,2010
9,경상남도,합계,2506393,1549690,705986,0,0,812336,1518322,31368,956703,광역단체장,2010


In [20]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      16 non-null     object
 1   구시군     16 non-null     object
 2   선거인수    16 non-null     int64 
 3   투표수     16 non-null     int64 
 4   보수정당    16 non-null     int64 
 5   진보정당    16 non-null     int64 
 6   그외정당    16 non-null     int64 
 7   무소속     16 non-null     int64 
 8   득표수_계   16 non-null     int64 
 9   무효투표수   16 non-null     int64 
 10  기권수     16 non-null     int64 
 11  선거종류    16 non-null     object
 12  선거년도    16 non-null     object
dtypes: int64(9), object(4)
memory usage: 1.8+ KB


In [21]:
# 수치형 열만 합계 구하기
summary_row = df_5.select_dtypes(include='number').sum().to_frame().T

# object 열 값 추가
summary_row.insert(0, '시도', '전국')
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '선거종류', '광역단체장')
summary_row.insert(0, '선거년도', '2010')


# summary_row를 맨 아래에 붙이기
df_5 = pd.concat([df_5, summary_row], ignore_index=True)

In [22]:
df_5['시도'].unique()

array(['부산광역시', '충청북도', '충청남도', '대구광역시', '대전광역시', '강원도', '광주광역시', '경상북도',
       '경기도', '경상남도', '인천광역시', '제주특별자치도', '전라북도', '전라남도', '서울특별시',
       '울산광역시', '전국'], dtype=object)

In [23]:
import pandas as pd

# 1. 지역명 매핑 딕셔너리 정의
region_mapping = {
    '부산광역시': '부산',
    '충청북도': '충북',
    '충청남도': '충남',
    '대구광역시': '대구',
    '대전광역시': '대전',
    '강원도': '강원',
    '광주광역시': '광주',
    '경상북도': '경북',
    '경기도': '경기',
    '경상남도': '경남',
    '인천광역시': '인천',
    '제주특별자치도': '제주',
    '전라북도': '전북',
    '전라남도': '전남',
    '서울특별시': '서울',
    '울산광역시': '울산',
    '전국': '전국'
}

# 2. 지역명 변경
df_5 = df_5.replace({'시도': region_mapping})

# 3. 원하는 행 순서 정의
desired_order = [
    '전국', '서울', '부산', '대구', '인천', '광주', '대전', '울산',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

# 4. 행 순서 변경
df_5 = df_5.set_index('시도').reindex(desired_order).reset_index()

In [24]:
df_5 = df_5.rename(columns={'시도': '지역'}).drop(columns=['구시군']).reset_index(drop=True)

In [25]:
df_5

,지역,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,전국,38851159,21162998,9441701,6314535,3864902,1031283,20652416,510577,17688161,광역단체장,2010
1,서울,8211461,4426182,2086127,2059715,251830,0,4397672,28510,3785279,광역단체장,2010
2,부산,2849895,1410126,770507,619565,0,0,1390072,20054,1439769,광역단체장,2010
3,대구,1928835,886035,633118,146458,88599,0,868175,17860,1042800,광역단체장,2010
4,인천,2096853,1067431,469040,556902,30838,0,1056780,10651,1029422,광역단체장,2010
5,광주,1064913,529901,74490,297003,151990,0,523483,6418,535012,광역단체장,2010
6,대전,1127547,596683,168616,137751,285196,0,591563,5120,530864,광역단체장,2010
7,울산,838805,462103,279421,0,176693,0,456114,5989,376702,광역단체장,2010
8,경기,8761840,4534771,2271492,0,2079892,0,4351384,183387,4227069,광역단체장,2010
9,강원,1190509,741724,326111,388443,0,0,714554,27170,448785,광역단체장,2010


## 6th_2014

In [26]:
# 예시: 모든 CSV에서 '구시군' 열의 값이 '합계'인 행들만 추출하여 결계
df_6 = extract_and_combine_csvs(
    username="sw1kwon",
    repo_name="korean-elections",
    folder_path="temp/v2_2_g/6th_2014",
    filter_column="구시군",        # 추출할 조건의 열 이름
    filter_value="합계",            # 그 열에서 찾을 값
    new_column=["선거종류", "선거년도"],        # 리스트
    new_column_value=["광역단체장", "2014"]       # 리스트 (같은 인덱스 대응)
)

📋 발견된 파일들: 17개
✅ temp2_2_governor_busan_6.csv: 1행 추출
✅ temp2_2_governor_chungbuk_6.csv: 1행 추출
✅ temp2_2_governor_chungnam_6.csv: 1행 추출
✅ temp2_2_governor_daegu_6.csv: 1행 추출
✅ temp2_2_governor_daejeon_6.csv: 1행 추출
✅ temp2_2_governor_gangwon_6.csv: 1행 추출
✅ temp2_2_governor_gwangju_6.csv: 1행 추출
✅ temp2_2_governor_gyeongbuk_6.csv: 1행 추출
✅ temp2_2_governor_gyeonggi_6.csv: 1행 추출
✅ temp2_2_governor_gyeongnam_6.csv: 1행 추출
✅ temp2_2_governor_incheon_6.csv: 1행 추출
✅ temp2_2_governor_jeju_6.csv: 1행 추출
✅ temp2_2_governor_jeonbuk_6.csv: 1행 추출
✅ temp2_2_governor_jeonnam_6.csv: 1행 추출
✅ temp2_2_governor_sejong_6.csv: 1행 추출
✅ temp2_2_governor_seoul_6.csv: 1행 추출
✅ temp2_2_governor_ulsan_6.csv: 1행 추출
📊 새로운 열 '선거종류' 추가 완료 (값: '광역단체장')
📊 새로운 열 '선거년도' 추가 완료 (값: '2014')

🎉 총 17행이 결합되었습니다.


In [27]:
df_6

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,부산광역시,합계,2932179,1629167,797926,0,0,777225,1575151,54016,1303012,광역단체장,2014
1,충청북도,합계,1261119,741049,346152,361115,18590,0,725857,15192,520070,광역단체장,2014
2,충청남도,합계,1644554,916206,392315,465994,0,34204,892513,23693,728348,광역단체장,2014
3,대구광역시,합계,2012579,1052638,581175,418891,23779,14774,1038619,14019,959941,광역단체장,2014
4,대전광역시,합계,1207972,652671,301389,322762,20355,0,644506,8165,555301,광역단체장,2014
5,강원도,합계,1255469,781359,369201,381338,15774,0,766313,15046,474110,광역단체장,2014
6,광주광역시,합계,1138418,650008,21614,367203,27985,217915,634717,15291,488410,광역단체장,2014
7,경상북도,합계,2211734,1314925,986989,189603,93067,0,1269659,45266,896809,광역단체장,2014
8,경기도,합계,9679317,5156691,2524981,2481824,0,0,5006805,149886,4522626,광역단체장,2014
9,경상남도,합계,2658347,1589673,913162,559367,79015,0,1551544,38129,1068674,광역단체장,2014


In [28]:
df_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      17 non-null     object
 1   구시군     17 non-null     object
 2   선거인수    17 non-null     int64 
 3   투표수     17 non-null     int64 
 4   보수정당    17 non-null     int64 
 5   진보정당    17 non-null     int64 
 6   그외정당    17 non-null     int64 
 7   무소속     17 non-null     int64 
 8   득표수_계   17 non-null     int64 
 9   무효투표수   17 non-null     int64 
 10  기권수     17 non-null     int64 
 11  선거종류    17 non-null     object
 12  선거년도    17 non-null     object
dtypes: int64(9), object(4)
memory usage: 1.9+ KB


In [29]:
# 수치형 열만 합계 구하기
summary_row = df_6.select_dtypes(include='number').sum().to_frame().T

# object 열 값 추가
summary_row.insert(0, '시도', '전국')
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '선거종류', '광역단체장')
summary_row.insert(0, '선거년도', '2014')


# summary_row를 맨 아래에 붙이기
df_6 = pd.concat([df_6, summary_row], ignore_index=True)

In [30]:
df_6['시도'].unique()

array(['부산광역시', '충청북도', '충청남도', '대구광역시', '대전광역시', '강원도', '광주광역시', '경상북도',
       '경기도', '경상남도', '인천광역시', '제주특별자치도', '전라북도', '전라남도', '세종특별자치시',
       '서울특별시', '울산광역시', '전국'], dtype=object)

In [31]:
import pandas as pd

# 1. 지역명 매핑 딕셔너리 정의
region_mapping = {
    '부산광역시': '부산',
    '충청북도': '충북',
    '충청남도': '충남',
    '대구광역시': '대구',
    '대전광역시': '대전',
    '강원도': '강원',
    '광주광역시': '광주',
    '경상북도': '경북',
    '경기도': '경기',
    '경상남도': '경남',
    '인천광역시': '인천',
    '제주특별자치도': '제주',
    '전라북도': '전북',
    '전라남도': '전남',
    '세종특별자치시': '세종',
    '서울특별시': '서울',
    '울산광역시': '울산',
    '전국': '전국'
}

# 2. 지역명 변경
df_6 = df_6.replace({'시도': region_mapping})

# 3. 원하는 행 순서 정의
desired_order = [
    '전국', '서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

# 4. 행 순서 변경
df_6 = df_6.set_index('시도').reindex(desired_order).reset_index()

In [32]:
df_6 = df_6.rename(columns={'시도': '지역'}).drop(columns=['구시군']).reset_index(drop=True)

In [33]:
df_6

,지역,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,전국,41296228,23462336,10735126,10384406,730351,1044118,22894001,568335,17833892,광역단체장,2014
1,서울,8441594,4948897,2109869,2752171,41241,0,4903281,45616,3492697,광역단체장,2014
2,부산,2932179,1629167,797926,0,0,777225,1575151,54016,1303012,광역단체장,2014
3,대구,2012579,1052638,581175,418891,23779,14774,1038619,14019,959941,광역단체장,2014
4,인천,2319198,1244502,615077,593555,22651,0,1231283,13219,1074696,광역단체장,2014
5,광주,1138418,650008,21614,367203,27985,217915,634717,15291,488410,광역단체장,2014
6,대전,1207972,652671,301389,322762,20355,0,644506,8165,555301,광역단체장,2014
7,울산,912325,511881,306311,0,161843,0,468154,43727,400444,광역단체장,2014
8,세종,101559,63629,26451,36203,0,0,62654,975,37930,광역단체장,2014
9,경기,9679317,5156691,2524981,2481824,0,0,5006805,149886,4522626,광역단체장,2014


## 7th_2018

In [34]:
# 예시: 모든 CSV에서 '구시군' 열의 값이 '합계'인 행들만 추출하여 결계
df_7 = extract_and_combine_csvs(
    username="sw1kwon",
    repo_name="korean-elections",
    folder_path="temp/v2_2_g/7th_2018",
    filter_column="구시군",        # 추출할 조건의 열 이름
    filter_value="합계",            # 그 열에서 찾을 값
    new_column=["선거종류", "선거년도"],        # 리스트
    new_column_value=["광역단체장", "2018"]       # 리스트 (같은 인덱스 대응)
)

📋 발견된 파일들: 17개
✅ temp2_2_governor_busan_7.csv: 1행 추출
✅ temp2_2_governor_chungbuk_7.csv: 1행 추출
✅ temp2_2_governor_chungnam_7.csv: 1행 추출
✅ temp2_2_governor_daegu_7.csv: 1행 추출
✅ temp2_2_governor_daejeon_7.csv: 1행 추출
✅ temp2_2_governor_gangwon_7.csv: 1행 추출
✅ temp2_2_governor_gwangju_7.csv: 1행 추출
✅ temp2_2_governor_gyeongbuk_7.csv: 1행 추출
✅ temp2_2_governor_gyeonggi_7.csv: 1행 추출
✅ temp2_2_governor_gyeongnam_7.csv: 1행 추출
✅ temp2_2_governor_incheon_7.csv: 1행 추출
✅ temp2_2_governor_jeju_7.csv: 1행 추출
✅ temp2_2_governor_jeonbuk_7.csv: 1행 추출
✅ temp2_2_governor_jeonnam_7.csv: 1행 추출
✅ temp2_2_governor_sejong_7.csv: 1행 추출
✅ temp2_2_governor_seoul_7.csv: 1행 추출
✅ temp2_2_governor_ulsan_7.csv: 1행 추출
📊 새로운 열 '선거종류' 추가 완료 (값: '광역단체장')
📊 새로운 열 '선거년도' 추가 완료 (값: '2018')

🎉 총 17행이 결합되었습니다.


In [35]:
df_7

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,부산광역시,합계,2939046,1727684,632806,940469,102801,26720,1702796,24888,1211362,광역단체장,2018
1,충청북도,합계,1318186,782316,227371,468750,70330,0,766451,15865,535870,광역단체장,2018
2,충청남도,합계,1740413,1011447,345577,615870,23012,0,984459,26988,728966,광역단체장,2018
3,대구광역시,합계,2047286,1172279,619165,458112,74955,0,1152232,20047,875007,광역단체장,2018
4,대전광역시,합계,1219513,706983,224306,393354,79622,0,697282,9701,512530,광역단체장,2018
5,강원도,합계,1296196,819193,282456,518447,0,0,800903,18290,477003,광역단체장,2018
6,광주광역시,합계,1172429,694252,34487,573995,74228,0,682710,11542,478177,광역단체장,2018
7,경상북도,합계,2251538,1457539,732785,482564,190659,0,1406008,51531,793999,광역단체장,2018
8,경기도,합계,10533027,6084955,2122433,3370621,482473,0,5975527,109428,4448072,광역단체장,2018
9,경상남도,합계,2765485,1819391,765809,941491,75418,0,1782718,36673,946094,광역단체장,2018


In [36]:
df_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      17 non-null     object
 1   구시군     17 non-null     object
 2   선거인수    17 non-null     int64 
 3   투표수     17 non-null     int64 
 4   보수정당    17 non-null     int64 
 5   진보정당    17 non-null     int64 
 6   그외정당    17 non-null     int64 
 7   무소속     17 non-null     int64 
 8   득표수_계   17 non-null     int64 
 9   무효투표수   17 non-null     int64 
 10  기권수     17 non-null     int64 
 11  선거종류    17 non-null     object
 12  선거년도    17 non-null     object
dtypes: int64(9), object(4)
memory usage: 1.9+ KB


In [37]:
# 수치형 열만 합계 구하기
summary_row = df_7.select_dtypes(include='number').sum().to_frame().T

# object 열 값 추가
summary_row.insert(0, '시도', '전국')
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '선거종류', '광역단체장')
summary_row.insert(0, '선거년도', '2018')


# summary_row를 맨 아래에 붙이기
df_7 = pd.concat([df_7, summary_row], ignore_index=True)

In [38]:
df_7['시도'].unique()

array(['부산광역시', '충청북도', '충청남도', '대구광역시', '대전광역시', '강원도', '광주광역시', '경상북도',
       '경기도', '경상남도', '인천광역시', '제주특별자치도', '전라북도', '전라남도', '세종특별자치시',
       '서울특별시', '울산광역시', '전국'], dtype=object)

In [39]:
import pandas as pd

# 1. 지역명 매핑 딕셔너리 정의
region_mapping = {
    '부산광역시': '부산',
    '충청북도': '충북',
    '충청남도': '충남',
    '대구광역시': '대구',
    '대전광역시': '대전',
    '강원도': '강원',
    '광주광역시': '광주',
    '경상북도': '경북',
    '경기도': '경기',
    '경상남도': '경남',
    '인천광역시': '인천',
    '제주특별자치도': '제주',
    '전라북도': '전북',
    '전라남도': '전남',
    '세종특별자치시': '세종',
    '서울특별시': '서울',
    '울산광역시': '울산',
    '전국': '전국'
}

# 2. 지역명 변경
df_7 = df_7.replace({'시도': region_mapping})

# 3. 원하는 행 순서 정의
desired_order = [
    '전국', '서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

# 4. 행 순서 변경
df_7 = df_7.set_index('시도').reindex(desired_order).reset_index()

In [40]:
df_7 = df_7.rename(columns={'시도': '지역'}).drop(columns=['구시군']).reset_index(drop=True)

In [41]:
df_7

,지역,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,전국,42907715,25832076,7959542,14191438,2980640,204975,25336595,495481,17075639,광역단체장,2018
1,서울,8380947,5019098,1158487,2619497,1183888,0,4961872,57226,3361849,광역단체장,2018
2,부산,2939046,1727684,632806,940469,102801,26720,1702796,24888,1211362,광역단체장,2018
3,대구,2047286,1172279,619165,458112,74955,0,1152232,20047,875007,광역단체장,2018
4,인천,2440779,1349084,470937,766186,91526,0,1328649,20435,1091695,광역단체장,2018
5,광주,1172429,694252,34487,573995,74228,0,682710,11542,478177,광역단체장,2018
6,대전,1219513,706983,224306,393354,79622,0,697282,9701,512530,광역단체장,2018
7,울산,942550,610698,240475,317341,42210,0,600026,10672,331852,광역단체장,2018
8,세종,222852,137603,24546,96896,14444,0,135886,1717,85249,광역단체장,2018
9,경기,10533027,6084955,2122433,3370621,482473,0,5975527,109428,4448072,광역단체장,2018


## 8th_2022

In [42]:
# 예시: 모든 CSV에서 '구시군' 열의 값이 '합계'인 행들만 추출하여 결계
df_8 = extract_and_combine_csvs(
    username="sw1kwon",
    repo_name="korean-elections",
    folder_path="temp/v2_2_g/8th_2022",
    filter_column="구시군",        # 추출할 조건의 열 이름
    filter_value="합계",            # 그 열에서 찾을 값
    new_column=["선거종류", "선거년도"],        # 리스트
    new_column_value=["광역단체장", "2022"]       # 리스트 (같은 인덱스 대응)
)

📋 발견된 파일들: 17개
✅ temp2_2_governor_busan_8.csv: 1행 추출
✅ temp2_2_governor_chungbuk_8.csv: 1행 추출
✅ temp2_2_governor_chungnam_8.csv: 1행 추출
✅ temp2_2_governor_daegu_8.csv: 1행 추출
✅ temp2_2_governor_daejeon_8.csv: 1행 추출
✅ temp2_2_governor_gangwon_8.csv: 1행 추출
✅ temp2_2_governor_gwangju_8.csv: 1행 추출
✅ temp2_2_governor_gyeongbuk_8.csv: 1행 추출
✅ temp2_2_governor_gyeonggi_8.csv: 1행 추출
✅ temp2_2_governor_gyeongnam_8.csv: 1행 추출
✅ temp2_2_governor_incheon_8.csv: 1행 추출
✅ temp2_2_governor_jeju_8.csv: 1행 추출
✅ temp2_2_governor_jeonbuk_8.csv: 1행 추출
✅ temp2_2_governor_jeonnam_8.csv: 1행 추출
✅ temp2_2_governor_sejong_8.csv: 1행 추출
✅ temp2_2_governor_seoul_8.csv: 1행 추출
✅ temp2_2_governor_ulsan_8.csv: 1행 추출
📊 새로운 열 '선거종류' 추가 완료 (값: '광역단체장')
📊 새로운 열 '선거년도' 추가 완료 (값: '2022')

🎉 총 17행이 결합되었습니다.


In [43]:
df_8

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,부산광역시,합계,2916832,1432194,938601,455901,19733,0,1414235,17959,1484638,광역단체장,2022
1,충청북도,합계,1368779,692324,395517,284166,0,0,679683,12641,676455,광역단체장,2022
2,충청남도,합계,1803096,898369,468658,401308,0,0,869966,28403,904727,광역단체장,2022
3,대구광역시,합계,2044579,883141,685159,156429,28446,0,870034,13107,1161438,광역단체장,2022
4,대전광역시,합계,1233557,612639,310035,295555,0,0,605590,7049,620918,광역단체장,2022
5,강원도,합계,1336080,772498,409461,347766,0,0,757227,15271,563582,광역단체장,2022
6,광주광역시,합계,1206886,454516,71062,334699,24414,0,446770,7746,752370,광역단체장,2022
7,경상북도,합계,2268707,1194595,904675,255775,0,0,1160450,34145,1074112,광역단체장,2022
8,경기도,합계,11497206,5820631,2818680,2827593,61778,54758,5762809,57822,5676575,광역단체장,2022
9,경상남도,합계,2804287,1497400,963473,431569,71286,0,1466328,31072,1306887,광역단체장,2022


In [44]:
df_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      17 non-null     object
 1   구시군     17 non-null     object
 2   선거인수    17 non-null     int64 
 3   투표수     17 non-null     int64 
 4   보수정당    17 non-null     int64 
 5   진보정당    17 non-null     int64 
 6   그외정당    17 non-null     int64 
 7   무소속     17 non-null     int64 
 8   득표수_계   17 non-null     int64 
 9   무효투표수   17 non-null     int64 
 10  기권수     17 non-null     int64 
 11  선거종류    17 non-null     object
 12  선거년도    17 non-null     object
dtypes: int64(9), object(4)
memory usage: 1.9+ KB


In [45]:
# 수치형 열만 합계 구하기
summary_row = df_8.select_dtypes(include='number').sum().to_frame().T

# object 열 값 추가
summary_row.insert(0, '시도', '전국')
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '선거종류', '광역단체장')
summary_row.insert(0, '선거년도', '2022')


# summary_row를 맨 아래에 붙이기
df_8 = pd.concat([df_8, summary_row], ignore_index=True)

In [46]:
df_8['시도'].unique()

array(['부산광역시', '충청북도', '충청남도', '대구광역시', '대전광역시', '강원도', '광주광역시', '경상북도',
       '경기도', '경상남도', '인천광역시', '제주특별자치도', '전라북도', '전라남도', '세종특별자치시',
       '서울특별시', '울산광역시', '전국'], dtype=object)

In [47]:
import pandas as pd

# 1. 지역명 매핑 딕셔너리 정의
region_mapping = {
    '부산광역시': '부산',
    '충청북도': '충북',
    '충청남도': '충남',
    '대구광역시': '대구',
    '대전광역시': '대전',
    '강원도': '강원',
    '광주광역시': '광주',
    '경상북도': '경북',
    '경기도': '경기',
    '경상남도': '경남',
    '인천광역시': '인천',
    '제주특별자치도': '제주',
    '전라북도': '전북',
    '전라남도': '전남',
    '세종특별자치시': '세종',
    '서울특별시': '서울',
    '울산광역시': '울산',
    '전국': '전국'
}

# 2. 지역명 변경
df_8 = df_8.replace({'시도': region_mapping})

# 3. 원하는 행 순서 정의
desired_order = [
    '전국', '서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

# 4. 행 순서 변경
df_8 = df_8.set_index('시도').reindex(desired_order).reset_index()

In [48]:
df_8 = df_8.rename(columns={'시도': '지역'}).drop(columns=['구시군']).reset_index(drop=True)

In [49]:
df_8

,지역,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,전국,44303449,22564394,11989460,9762313,371202,73896,22213466,350928,21739055,광역단체장,2022
1,서울,8378339,4455161,2608277,1733183,66459,9000,4416919,38242,3923178,광역단체장,2022
2,부산,2916832,1432194,938601,455901,19733,0,1414235,17959,1484638,광역단체장,2022
3,대구,2044579,883141,685159,156429,28446,0,870034,13107,1161438,광역단체장,2022
4,인천,2534338,1240469,634250,545885,45000,0,1225135,15334,1293869,광역단체장,2022
5,광주,1206886,454516,71062,334699,24414,0,446770,7746,752370,광역단체장,2022
6,대전,1233557,612639,310035,295555,0,0,605590,7049,620918,광역단체장,2022
7,울산,941189,491866,290563,195430,0,0,485993,5873,449323,광역단체장,2022
8,세종,292259,149751,78415,69995,0,0,148410,1341,142508,광역단체장,2022
9,경기,11497206,5820631,2818680,2827593,61778,54758,5762809,57822,5676575,광역단체장,2022


# Merge

In [50]:
df_32 = pd.concat([df_3, df_4, df_5, df_6, df_7, df_8], ignore_index=True)
df_32.head()

,지역,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수,선거종류,선거년도
0,전국,34744232,16973662,8820102,4874653,1761560,1226757,16683072,290590,17770570,광역단체장,2002
1,서울,7665343,3510898,1819057,1496754,128981,34313,3479105,31793,4154445,광역단체장,2002
2,부산,2784721,1163040,729589,221938,192594,0,1144121,18919,1621681,광역단체장,2002
3,대구,1814278,751994,452943,0,0,287396,740339,11655,1062284,광역단체장,2002
4,인천,1809907,711602,393932,225210,82111,0,701253,10349,1098305,광역단체장,2002


In [51]:
df_32.to_csv("temp3_2_governor.csv", index=False, encoding="utf-8-sig")